# 1. 준비

In [1]:
#glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import glob 
import tensorflow as tf
import numpy as np
import os
import re 
from sklearn.model_selection import train_test_split

# 2. 데이터 읽어오기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path) 
raw_corpus = [] 

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        
        raw = f.read().splitlines()
        raw_corpus.extend(raw) 

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['', '', '[Spoken Intro:]']


# 3. 데이터 정제

## preprocess_sentence 로 정제

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [4]:
# 정제된 문장 모으는곳
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10]

['<start> spoken intro <end>',
 '<start> you ever want something <end>',
 '<start> that you know you shouldn t have <end>',
 '<start> the more you know you shouldn t have it , <end>',
 '<start> the more you want it <end>',
 '<start> and then one day you get it , <end>',
 '<start> it s so good too <end>',
 '<start> but it s just like my girl <end>',
 '<start> when she s around me <end>',
 '<start> i just feel so good , so good <end>']

# 4. 평가 데이터셋 분리

### 훈련/평가 데이터 분리하기 

1. tokenize => 데이터 tensor로 변환 후
2. sklearn train_test_split에 분리 -> train and validation
- 단어 크기 12,000 이상 / 20% 평가셋

In [5]:
# tokenize -> data to tensor

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)   
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=20)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 2701 2584 ...    0    0    0]
 [   2    7  156 ...    0    0    0]
 [   2   17    7 ...    0    0    0]
 ...
 [   2  311    1 ...    0    0    0]
 [   2  735    5 ...    0    0    0]
 [   2  735    5 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f7fe428bee0>


In [6]:
# 예제 추가정보 : pad
src_input = tensor[:, :-1]  

tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2 2701 2584    3    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
[2701 2584    3    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


In [7]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words + 1   


dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

In [8]:
# train / validation split

enc_train, enc_val, dec_train, dec_val = train_test_split(
    src_input, 
    tgt_input, 
    test_size =0.2, 
    shuffle = True, 
    random_state = 17)

In [9]:
print(enc_train.shape)

(140599, 19)


In [10]:
print(dec_train.shape)

(140599, 19)


# 5. 인공지능 만들기

- Embedding Size / Hidden size 조절
- Epoch : 10 안에 val_loss 2.2수준으로 하는 모델 설계

- model.fit() 사용
- epochs 사용

- val_loss 더 떨어뜨리기위해 model.fit()안에 다른 인자도 사용



In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

# 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기
embedding_size = 256 
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) 

In [12]:
optimizer = tf.keras.optimizers.Adam() 


# Loss
# 클래스 분류 문제에서 softmax 함수를 거치면 from_logits = False(default값),그렇지 않으면 from_logits = True.
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, 
    reduction='none' 
)

In [13]:
# 모델을 학습시키키 위한 학습과정을 설정하는 단계
# 손실함수와 훈련과정을 설정
model.compile(loss=loss, optimizer=optimizer) 


model.fit(dataset, epochs=10) 

Epoch 1/10
686/686 [==============================] - 168s 236ms/step - loss: 2.8419
Epoch 2/10
686/686 [==============================] - 163s 237ms/step - loss: 2.4557
Epoch 3/10
686/686 [==============================] - 163s 237ms/step - loss: 2.3027
Epoch 4/10
686/686 [==============================] - 163s 237ms/step - loss: 2.1912
Epoch 5/10
686/686 [==============================] - 163s 237ms/step - loss: 2.0966
Epoch 6/10
686/686 [==============================] - 163s 237ms/step - loss: 2.0095
Epoch 7/10
686/686 [==============================] - 163s 237ms/step - loss: 1.9275
Epoch 8/10
686/686 [==============================] - 163s 238ms/step - loss: 1.8505
Epoch 9/10
686/686 [==============================] - 163s 238ms/step - loss: 1.7783
Epoch 10/10
686/686 [==============================] - 163s 238ms/step - loss: 1.7084


In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): 

    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]


    while True: 
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""

    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated 

In [19]:
# 가사제출

generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i love you so much , i love you <end> '